In [5]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tnrange, tqdm_notebook
import random
from boardgamegeek import BoardGameGeek, BoardGameGeekAPIError
from sklearn.cluster import KMeans
from collections import Counter
from datetime import date
from retrying import retry
from collections import Counter
from time import time, strftime
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

pd.options.display.max_columns = 100

In [2]:
def get_games_of_publisher(games, publisher, query):
    print 'Processing games by {}'.format(publisher)
    more_games_available = True
    url = "https://boardgamegeek.com/search/boardgame/page/1?sort=rank&advsearch=1" + query
    while more_games_available:
        pagetext=requests.get(url)
        print url
        time.sleep(1)
        soup = BeautifulSoup(pagetext.text, "html.parser")
        if len(soup.find("div", attrs={"id": "collection"}).find_all("tr", attrs={"id":"row_"})) > 0:
            for gamesoup in soup.find("div", attrs={"id": "collection"}).find_all("tr", attrs={"id":"row_"}):
                game = {}
                try:
                    game['rank'] = int(gamesoup.find("td",attrs={"class":"collection_rank"}).find("a")['name'])
                except:
                    game['rank'] = 999999
                game['title'] = gamesoup.find("td",{"class":"collection_objectname"}).find("a").contents[0]
                game['url'] = "http://www.boardgamegeek.com" \
                    + gamesoup.find("td",{"class":"collection_objectname"}).find("a")['href']
                
                game['id'] = int(re.search('\/(\d*)\/', gamesoup.find("td",{"class":"collection_objectname"})\
                                                                         .find("a")['href']).group(1))
                try:
                    game['year'] = int(re.search('\((\d*)\)', gamesoup.find("td",{"class":"collection_objectname"})\
                                                                      .find("span").contents[0])
                                     .group(1))
                except:
                    game['year'] = 1900
                ratings = gamesoup.find_all("td",{"class":"collection_bggrating"})
                try:
                    game['bggrating'] = float(ratings[0].contents[0])
                except:
                    game['bggrating'] = None
                try:
                    game['avgrating'] = float(ratings[1].contents[0])
                except:
                    game['avgrating'] = None
                try:
                    game['votes'] = int(ratings[2].contents[0])
                except:
                    game['votes'] = 0
                game['publisher'] = publisher
                
                if game['id'] not in [g['id'] for g in games]:
                    games.append(game)
        try:
            url = "https://boardgamegeek.com" + soup.find("a",{"title":"next page"})['href']
        except:
            more_games_available = False
    print ''
    return games

In [3]:
games = []

In [5]:
publishers = {
     "999 Games": "&include%5Bpublisherid%5D=267",
     "Asmodee": "&include%5Bpublisherid%5D=157",
     "Avalon Hill": "&include%5Bpublisherid%5D=5",
     "Bergsala Enigma": "&include%5Bpublisherid%5D=6784",
     "Cool mini or not": "&include%5Bpublisherid%5D=34793",
     "Czech Games Edition": "&include%5Bpublisherid%5D=7345",
     "Days of Wonder": "&include%5Bpublisherid%5D=1027",
     "Don & Co": "&include%5Bpublisherid%5D=137"
     "Fantasy Flight Games": "&include%5Bpublisherid%5D=17",
     "Fun Forge": "&include%5Bpublisherid%5D=8832",
     "Games Workshop": "&include%5Bpublisherid%5D=26",
     "Guillotine Games": "&include%5Bpublisherid%5D=21020",
     "Hurrican": "&include%5Bpublisherid%5D=6015",
     "Iello": "&include%5Bpublisherid%5D=8923",
     "Intrafin Games": "&include%5Bpublisherid%5D=5380",
     "Libellud": "&include%5Bpublisherid%5D=9051",
     "Ludonaute": "&include%5Bpublisherid%5D=11688",
     "Monolith Games": "&include%5Bpublisherid%5D=27147",
     "Osprey Games": "&include%5Bpublisherid%5D=29313",
     "Plaid Hat Games": "&include%5Bpublisherid%5D=10754",
     "Queen Games": "&include%5Bpublisherid%5D=47",
     "Repos Production": "&include%5Bpublisherid%5D=4384",
     "Space Cowboys": "&include%5Bpublisherid%5D=25842",
     "Steve Jackson Games": "&include%5Bpublisherid%5D=19",
     "Story Factory": "&include%5Bpublisherid%5D=17940",
     "Studio McVey": "&include%5Bpublisherid%5D=21608",
     "The Game Master": "&include%5Bpublisherid%5D=2862",
     "White Goblin Games": "&include%5Bpublisherid%5D=4932",
     "Z-Man Games": "&include%5Bpublisherid%5D=538",
    }
for k, v in tqdm_notebook(publishers.iteritems(), total=len(publishers.keys())):
    games = get_games_of_publisher(games, k, v)

Processing games by Asmodee
https://boardgamegeek.com/search/boardgame/page/1?sort=rank&advsearch=1&include%5Bpublisherid%5D=157
https://boardgamegeek.com/search/boardgame/page/2?sort=rank&advsearch=1&include%5Bpublisherid%5D=157&sortdir=asc
https://boardgamegeek.com/search/boardgame/page/3?sort=rank&advsearch=1&include%5Bpublisherid%5D=157&sortdir=asc
https://boardgamegeek.com/search/boardgame/page/4?sort=rank&advsearch=1&include%5Bpublisherid%5D=157&sortdir=asc
https://boardgamegeek.com/search/boardgame/page/5?sort=rank&advsearch=1&include%5Bpublisherid%5D=157&sortdir=asc
https://boardgamegeek.com/search/boardgame/page/6?sort=rank&advsearch=1&include%5Bpublisherid%5D=157&sortdir=asc

Processing games by Don & Co
https://boardgamegeek.com/search/boardgame/page/1?sort=rank&advsearch=1&include%5Bpublisherid%5D=137

Processing games by Fun Forge
https://boardgamegeek.com/search/boardgame/page/1?sort=rank&advsearch=1&include%5Bpublisherid%5D=8832

Processing games by Czech Games Edition
h

In [7]:
df = pd.DataFrame(games).set_index('id').sort_values('rank')
df.to_csv('albion_boardgames.csv', encoding='utf-8')
df.head()

,avgrating,bggrating,publisher,rank,title,url,votes,year
id,,,,,,,,
161936,8.67,8.493,Z-Man Games,1,Pandemic Legacy: Season 1,http://www.boardgamegeek.com/boardgame/161936/...,19079,2015
182028,8.71,8.312,Czech Games Edition,2,Through the Ages: A New Story of Civilization,http://www.boardgamegeek.com/boardgame/182028/...,7916,2015
120677,8.29,8.150,White Goblin Games,5,Terra Mystica,http://www.boardgamegeek.com/boardgame/120677/...,25001,2012
187645,8.52,8.150,Fantasy Flight Games,6,Star Wars: Rebellion,http://www.boardgamegeek.com/boardgame/187645/...,9353,2016
167791,8.39,8.120,Intrafin Games,7,Terraforming Mars,http://www.boardgamegeek.com/boardgame/167791/...,12186,2016


## Add detailed data

In [58]:
def process_gameinfo(gameinfo_dict, gamedict, gid):
    g = bgg.game(game_id=gid)
    gamedict['description'] = g.description
    gamedict['minage'] = g.min_age
    gamedict['minplayers'] = g.min_players
    gamedict['maxplayers'] = g.max_players
    try:
        gamedict['minplaytime'] = int(gameinfo_dict['item']['minplaytime'])
    except:
        gamedict['minplaytime'] = np.nan
    try:
        gamedict['maxplaytime'] = int(gameinfo_dict['item']['maxplaytime'])
    except:
        gamedict['maxplaytime'] = np.nan
    try:
        gamedict['subdomain'] = gameinfo_dict['item']['rankinfo'][1]['subdomain']
    except:
        gamedict['subdomain'] = ''
    try:
        polls = gameinfo_dict['item']['polls']
        try:
            gamedict['weight'] = polls['boardgameweight']['averageweight']
        except:
            gamedict['weight'] = np.nan
        try:
            gamedict['weight_votes'] = polls['boardgameweight']['votes']
        except:
            gamedict['weight_votes'] = np.nan
        try:
            gamedict['suggested_age'] = int(polls['playerage'].replace('+',''))
        except:
            gamedict['suggested_age'] = np.nan
        try:
            gamedict['nplayers_best_min'] = int(min([d['min'] for d in polls['userplayers']['best']]))
        except:
            gamedict['nplayers_best_min'] = 0
        try:
            gamedict['nplayers_best_max'] = int(max([d['max'] for d in polls['userplayers']['best']]))
        except:
            gamedict['nplayers_best_max'] = 0
        try:
            gamedict['nplayers_recom_min'] = int(min([d['min'] for d in polls['userplayers']['recommended']]))
        except:
            gamedict['nplayers_recom_min'] = 0
        try:
            gamedict['nplayers_recom_max'] = int(max([d['max'] for d in polls['userplayers']['recommended']]))
        except:
            gamedict['nplayers_recom_max'] = 0
        try:
            gamedict['nplayers_votes'] = int(polls['userplayers']['totalvotes'])
        except:
            gamedict['nplayers_votes'] = np.nan
    except:
        gamedict['weight'] = weight
        gamedict['weight_votes'] = weight_votes
        gamedict['suggested_age'] = suggested_age
        gamedict['nplayers_best_min'] = n_players_best_min
        gamedict['nplayers_best_max'] = n_players_best_max
        gamedict['nplayers_recom_min'] = n_players_recom_min
        gamedict['nplayers_recom_max'] = n_players_recom_max
        gamedict['nplayers_votes'] = n_players_votes
    try:
        stats = gameinfo_dict['item']['stats']
        gamedict.update(stats)
    except:
        pass
    categories = {'cat-'+ele.lower(): 1 for ele in g.categories}
    gamedict.update(categories)
    mechanisms = {'mech-'+ele.lower(): 1 for ele in g.mechanics}
    gamedict.update(mechanisms)
    gamedict.update(categories)
    gamedict['updated'] = 1
    return gamedict

In [59]:
def retry_if_attribute_error(exception):
    """Return True if we should retry (in this case when it's an IOError), False otherwise"""
    print("             {} - URL timed out, so I'm going to retry.".format(strftime('%d %b, %H:%M:%S')))
    if isinstance(exception, AttributeError) or isinstance(exception, BoardGameGeekAPIError):
        return True
    else:
        print("             Encountered a new error: {}".format(exception))
        return False

@retry(wait_exponential_multiplier=500, wait_exponential_max=15000, stop_max_delay=30000,
       retry_on_exception=retry_if_attribute_error)
def update_gamedata(gamedict, gid):
    url = gamedict['url']
    pagetext = requests.get(url)
    soup = BeautifulSoup(pagetext.text, "html.parser")
    gameinfo = re.search('geekitemPreload = (\{.*\}\});', soup.find("script").contents[0]).group(1)
    gameinfo_dict = eval(gameinfo.replace("true","True")\
                                 .replace("false","False")\
                                 .replace("null","np.nan"))
    gamedict = process_gameinfo(gameinfo_dict, gamedict, gid)
    return gamedict

def save_data(games_dict):
    df = pd.DataFrame(games_dict).T.sort_values('rank')
    df['random'] = np.random.uniform(low=-0.5, high=0.5, size=(len(df),))
    df.to_csv('albion_boardgames.csv', encoding='utf-8', index_label='id')
    print("             Data saved.")
    return df

In [72]:
df = pd.read_csv('albion_boardgames.csv').set_index('id')
bgg = BoardGameGeek()
games_dict = df.T.to_dict()

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (84,159,164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
t0 = time()
for gid, row in tqdm_notebook(df.iterrows(),total=len(df)):
    if 'updated' in games_dict[gid]:
        if games_dict[gid]['updated'] == 1:
            continue
    try:
        print '{:>6} - {}'.format(gid, games_dict[gid]['url'])
    except:
        print "Couldn't print the game title for some reason"
    updated_gamedata = update_gamedata(games_dict[gid], gid)
    if isinstance(updated_gamedata, dict):
        games_dict[gid] = updated_gamedata
    else:
        print("Uh oh!")
        print(updated_gamedata)
    t1 = time()
    if t1 - t0 >= 120:
        df = save_data(games_dict)
        t0 = time()
df = save_data(games_dict)

 67185 - http://www.boardgamegeek.com/boardgame/67185/sobek
 19764 - http://www.boardgamegeek.com/boardgame/19764/punct
176558 - http://www.boardgamegeek.com/boardgame/176558/mafia-de-cuba
182189 - http://www.boardgamegeek.com/boardgame/182189/treasure-hunter
154182 - http://www.boardgamegeek.com/boardgame/154182/helios
 73070 - http://www.boardgamegeek.com/boardgame/73070/olympus
163370 - http://www.boardgamegeek.com/boardgame/163370/carcassonne-gold-rush
 76150 - http://www.boardgamegeek.com/boardgame/76150/revolver
175621 - http://www.boardgamegeek.com/boardgame/175621/epic-card-game
    28 - http://www.boardgamegeek.com/boardgame/28/illuminati-deluxe-edition
173101 - http://www.boardgamegeek.com/boardgame/173101/council-4
143404 - http://www.boardgamegeek.com/boardgame/143404/castellion
 54307 - http://www.boardgamegeek.com/boardgame/54307/chronicle
  4192 - http://www.boardgamegeek.com/boardgame/4192/warhammer
153507 - http://www.boardgamegeek.com/boardgame/153507/terra
148261 - h

### Clustering

In [74]:
for k,i in enumerate(list(df.columns)):
    print("{:>3} - {}".format(k, i))

  0 - average
  1 - avgrating
  2 - avgweight
  3 - baverage
  4 - bggrating
  5 - cat-abstract strategy
  6 - cat-action / dexterity
  7 - cat-adventure
  8 - cat-age of reason
  9 - cat-american civil war
 10 - cat-american indian wars
 11 - cat-american revolutionary war
 12 - cat-american west
 13 - cat-ancient
 14 - cat-animals
 15 - cat-arabian
 16 - cat-aviation / flight
 17 - cat-bluffing
 18 - cat-book
 19 - cat-card game
 20 - cat-children's game
 21 - cat-city building
 22 - cat-civil war
 23 - cat-civilization
 24 - cat-collectible components
 25 - cat-comic book / strip
 26 - cat-deduction
 27 - cat-dice
 28 - cat-economic
 29 - cat-educational
 30 - cat-electronic
 31 - cat-environmental
 32 - cat-expansion for base-game
 33 - cat-exploration
 34 - cat-fan expansion
 35 - cat-fantasy
 36 - cat-farming
 37 - cat-fighting
 38 - cat-game system
 39 - cat-horror
 40 - cat-humor
 41 - cat-industry / manufacturing
 42 - cat-mafia
 43 - cat-math
 44 - cat-mature / adult
 45 - ca

In [75]:
cat_cols = [col for col in df.columns if 'cat-' in col and col != 'cat-expansion for base-game' and col != 'cat-fan expansion']
mech_cols = [col for col in df.columns if 'mech-' in col]

import operator
from sklearn.cluster import AffinityPropagation

In [76]:
%%time

selection = (df['updated'] == 1) & (df['rank'] <= 1000)

for c,x in zip(['category-cluster','mechanics-cluster'],[cat_cols,mech_cols]):
    data = np.array(df[selection][x].fillna(0))

    print ''
    print 'Kmeans Clustering'
    print ''
    n_clusters = 5
    
    biggest_cluster = 999999
    imax = 50000
    i = 0
    cluster_max = 140
    while biggest_cluster >= cluster_max and i < imax:
        km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=1000, n_init=1).fit(data)
        clusters = list(km.predict(data))
        biggest_cluster = max([tupl[1] for tupl in Counter(clusters).items()])
        i += 1
    if i == imax and biggest_cluster >= cluster_max:
        print 'No suitable cluster configuration found!'
        
    print Counter(clusters).items()

    cluster_titles = []
    for cl in range(0,n_clusters):
        center = km.cluster_centers_[cl]
        center = {x[idx]:int(np.round(cat*100)) for idx, cat in enumerate(center)}
        sorted_x = sorted(center.items(), key=operator.itemgetter(1), reverse=True)
        for cat in sorted_x[:5]:
            print '* {} - {}'.format(re.sub(r'(cat-|mech-)','',cat[0]),cat[1])
        
        cluster_titles.append(str(cl+1) + ') ' + ' - '.join([re.sub(r'(cat-|mech-)','',cat[0]) for cat in sorted_x[:3]]))
        indices = [i for i, clu in enumerate(clusters) if clu == cl]
        df_cluster = df[selection].iloc[indices]
        print ''
        print list(df_cluster[df_cluster['votes'] >= 10000]['title'])
        print '-'*100 + '\n'
       
    print cluster_titles
    
    
    data_all = np.array(df[x].fillna(0))
    clusters_all = list(km.predict(data_all))
    print len(df)
    print len([cluster_titles[ele] for ele in clusters_all])
    df[c] = [cluster_titles[ele] for ele in clusters_all]
df.head()


Kmeans Clustering

[(0, 126), (1, 95), (2, 88), (3, 113), (4, 119)]
* medieval - 25
* wargame - 19
* city building - 17
* ancient - 15
* fantasy - 13

['Twilight Imperium (Third Edition)', 'Five Tribes', 'El Grande', 'Tigris & Euphrates', 'A Game of Thrones: The Board Game (Second Edition)', 'Ticket to Ride: Europe', 'Village', "Memoir '44", 'Ticket to Ride', 'Ra', 'Carcassonne', "Sid Meier's Civilization: The Board Game", 'Samurai', 'Kingsburg', 'Catan', 'Thurn and Taxis', 'Alhambra', 'Jamaica', 'Tokaido', 'Kingdom Builder', 'Roll Through the Ages: The Bronze Age']
----------------------------------------------------------------------------------------------------

* economic - 100
* card game - 22
* medieval - 19
* city building - 18
* industry / manufacturing - 18

['Terra Mystica', 'Terraforming Mars', 'Caverna: The Cave Farmers', 'Agricola', 'Through the Ages: A Story of Civilization', 'Power Grid', 'Brass: Lancashire', 'Le Havre', "Tzolk'in: The Mayan Calendar", 'Concordia', 'Ca

In [77]:
df.to_csv('albion_boardgames.csv', encoding='utf-8', index_label='id')

In [78]:
print date.today().year
len(df)

2017


5364

In [79]:
def final_score(row):
    years_since_publishing = date.today().year - row['year'] + 2
    lasting_popularity = np.log10(row['numplays_month']+row['numplays']/10000. * np.log10(years_since_publishing))
    final_score = row['bggrating']*2+lasting_popularity
    return final_score

In [80]:
def best_players(row):
    if row['nplayers_best_min'] == 2:
        return '2 players'
    elif row['nplayers_best_min'] == 3:
        return '3 players'
    elif row['nplayers_best_min'] == 4:
        return '4 players'
    elif row['nplayers_best_min'] >= 5:
        return '5 or more players'

In [81]:
df['final_score'] = df.apply(final_score, axis=1)
df['best_for'] = df.apply(best_players, axis=1)

TypeError: ("unsupported operand type(s) for /: 'str' and 'float'", u'occurred at index 67185')

In [ ]:
#best_players_selections = ['2 players', '3 or 4 players', '5 or more players']
#weight_selections = [(1,1.5), (1.5,2), (2,2.5), (2.5,3.2), (3.2,4)]
best_players_selections = ['2 players', '3 players', '4 players', '5 or more players']
weight_selections = [(1,1.4), (1.5,1.9), (1.9,2.5), (2.5,3.1), (3.1,4)]

cols = ['title', 'year', 'weight', 'best_for', 'final_score', 'category-cluster', 
        'mechanics-cluster', 'url']

for n_players in best_players_selections:
    for weight_range in weight_selections:
        selection = (df['best_for'] == n_players) & (df['weight'] >= weight_range[0]) & \
            (df['weight'] <= weight_range[1]) & (df['year'] >= 2000) & (df['weight_votes'] >= 10) & \
            (df['final_score'] >= 15)
        print 'Weight: {} to {} - Best for {}'.format(weight_range[0], weight_range[1], n_players)
        print '-'*80
        selected = df[selection][cols].sort_values('final_score', ascending=False).head(10)
        for idx, row in selected.iterrows():
            print '{:<35} ({}) - {:.2f}, {}, {:.1f}, Cat: {}, Mech: {}\n\t\t{}'\
                .format(row['title'][:35], row['year'], row['weight'], row['best_for'], row['final_score'],
                        row['category-cluster'][0], row['mechanics-cluster'][0], row['url'])
        print '\n\n'

In [ ]:
df.columns

In [ ]:
df[df['weight'] <= 1.25].sort_values('numplays_month', ascending=False)[['title','weight','numplays_month']]

### Topic modelling

In [ ]:
len(df)

In [ ]:
def process_documents(documents):
    documents = [unicode(str(d), errors='ignore') for d in documents]
    texts = [document.lower().split() for document in documents]
    return texts

In [ ]:
class MyCorpus(object):
    def __init__(self, directory='models', filename='output'):
        self.dictionary = None
        self.corpus = None
        if not os.path.exists(directory):
            os.makedirs(directory)
        self.dict_path = '{0}/{1}.dict'.format(directory, filename)
        self.corpus_path = '{0}/{1}.mm'.format(directory, filename)

    def get_dict(self, documents):
        dictionary = corpora.Dictionary(process_documents(documents))
        stoplist = set('a'.split())
        stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
        once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq < 3]
        dictionary.filter_tokens(stop_ids + once_ids)
        dictionary.compactify() # remove gaps in id sequence after words that were removed
        self.dictionary = dictionary

    def get_corpus(self, documents):
        self.corpus = [self.dictionary.doc2bow(text) for text in process_documents(documents)]
        
    def save_dict(self):
        self.dictionary.save(self.dict_path)
        
    def save_corpus(self):
        corpora.MmCorpus.serialize(self.corpus_path, self.corpus)
        
    def load_corpus(self):
        self.corpus = corpora.MmCorpus(self.corpus_path)
        
    def load_dictionary(self):
        self.dictionary = corpora.Dictionary.load(self.dict_path)
        
class TFIDFModel(MyCorpus): 
    def __init__(self, directory='models', filename='output'):
        super(TFIDFModel, self).__init__(directory='models', filename='output')
        self.tfidf = None
        self.tfidf_path = '{0}/{1}.tfidf_model'.format(directory, filename)
        
    def train_tfidf(self):
        self.tfidf = models.TfidfModel(self.corpus)
        
    def save_tfidf(self):
        self.tfidf.save(self.tfidf_path)
        
    def load_tfidf(self):
        self.tfidf = models.TfidfModel.load(self.tfidf_path)
        
class LSITFIDFModel(TFIDFModel):
    def __init__(self, directory='models', filename='output'):
        super(LSITFIDFModel, self).__init__(directory='models', filename='output')
        self.model = None
        self.model_path = '{0}/{1}.lsi_tfidf'.format(directory, filename)
    
    def train_lsi(self, ntopics = 2):
        self.model = models.LsiModel(self.tfidf[self.corpus], id2word=self.dictionary, num_topics=ntopics, distributed=False) 
    
    def load_lsi(self):
        self.model = models.LsiModel.load(self.model_path)
        
    def save_lsi(self):
        self.model.save(self.model_path)

In [ ]:
documents = list(df[df['weight'] > 0]['description'])
print len(documents)

In [ ]:
import os
import Pyro4
from gensim import corpora, models, similarities

In [ ]:
lsi = LSITFIDFModel()
lsi.get_dict(documents)
lsi.save_dict()

In [ ]:
lsi.get_corpus(documents)
lsi.save_corpus()

In [ ]:
lsi.train_tfidf()
lsi.save_tfidf()

In [ ]:
%%time
lsi.train_lsi(ntopics=10)

In [ ]:
corpus = [lsi.dictionary.doc2bow(text) for text in process_documents(documents)]
trans_corpus = lsi.tfidf[corpus]
lsi_vec = lsi.model[trans_corpus]

In [ ]:
values_list = []
for vec in lsi_vec:
    values = []
    for tupl in vec:
        values.append(tupl[1])
    values_list.append(values)

In [ ]:
data = np.array(values_list)
data.shape

In [ ]:
n_clusters = 20

km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=1000, n_init=1)
km.fit(data)

In [ ]:
clusters = list(km.predict(data))
print len(clusters)

In [ ]:
for c in range(0,n_clusters):
    indices = [i for i, x in enumerate(clusters) if x == c]
    indices
    df_cluster = df[df['weight'] > 0].iloc[indices]
    print list(df_cluster[df_cluster['votes'] >= 10000]['title'])
    print '-'*100

In [ ]:
Counter(clusters).items()